# Notebook to read and display optimization results

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from pulp import *
import sys
from utilities import import_excel
import plotly.graph_objects as go
import plotly.express as px
import pickle
from plots import plot_ts_optim, plot_pie_energy, plot_storage, plot_stack_production

Choose country to display

In [2]:
country_name = 'California'
year = 2021
curtail_max= 0.01

In [3]:
data = pd.read_pickle(f"results/{country_name}/optimization_results_curtail_{curtail_max}_mean_load.pickle")

In [4]:
optimized_pv = data['optimized_pv']
optimized_wind = data['optimized_wind']
optimized_charge = data['optimized_charge']
optimized_discharge = data['optimized_discharge']
optimized_p_curt = data['optimized_p_curt']
optimized_stock = data['optimized_stock']
optimized_dispatchable = data['optimized_dispatchable']
Load_ts = data['consumption']

E_wind = data['E_wind']/1000000
E_pv = data['E_pv']/1000000
E_dispatch = data['E_dispatch']/1000000
E_curt = data['E_curt']/1000000
E_loss  = data['E_loss']/1000000
E_stock = data['E_stock']/1000000
E_destock = data['E_destock']/1000000

In [5]:
print(f'Total wind energy: {E_wind} TWh' )
print(f'Total solar energy: {E_pv} TWh' )
print(f'Total dispatchable energy: {E_dispatch} TWh' )
print(f'Total stored energy: {E_stock} TWh' )
print(f'Total energy supplied by storage: {E_destock} TWh' )
print(f'Total lost energy: {E_loss} TWh' )
print(f'Total curtailed energy: {E_curt} TWh' )

Total wind energy: 36.65662707226 TWh
Total solar energy: 117.47531490896856 TWh
Total dispatchable energy: 73.57746977346251 TWh
Total stored energy: 42.38822232309331 TWh
Total energy supplied by storage: 33.91057788201005 TWh
Total lost energy: 8.477644441083262 TWh
Total curtailed energy: 0.050277809618607956 TWh


In [13]:
colors_dict = {
    'Wind': 'steelblue',        
    'PV': 'gold',
    'Discharge': 'orangered',    
    'SOC': 'darkgreen',           
    'Charge': 'purple',
    'Consumption': 'green',          
    'Dispatchable': 'crimson',       
    'Curtailment': 'cyan'    
}

In [14]:
plot_ts_optim([optimized_pv, optimized_wind, optimized_dispatchable, optimized_p_curt,np.array(Load_ts) ], ['PV', 'Wind', 'Dispatchable', 'Curtailment', 'Consumption'], country_name, colors_dict = colors_dict, savefig=False)

In [15]:
plot_storage(optimized_charge, optimized_discharge, optimized_stock, country_name, colors_dict = colors_dict, savefig=True)

In [ ]:
plot_stack_production(optimized_pv, optimized_wind, optimized_dispatchable, country_name, colors = colors_dict)

In [ ]:
plot_pie_energy([E_wind, E_pv, E_dispatch], country_name, colors_dict = colors_dict, savefig=False)

In [ ]:

fig = go.Figure(data=[
    go.Bar(name='Renewables', x=[country_name], y=[E_wind+E_pv]),
    go.Bar(name='From stock', x=[country_name], y=[E_destock]),
    go.Bar(name='Dispatchable', x=[country_name], y=[E_dispatch]),
    go.Bar(name='Curtailment', x=[country_name], y=[-E_curt]),
    go.Bar(name='Loss', x=[country_name], y=[-E_loss])
])

fig.update_layout(
    title='Energy production',
    barmode='relative',
    yaxis_title = 'TWh'
)

fig.show()